In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import folium

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

import sys
import os

random_state = 10

backend_path = os.path.abspath('../backend')
sys.path.append(backend_path)
from get_metrics import get_metrics_regression, get_metrics_classification
from check_overfitting import check_overfitting

In [ ]:
# Чтение DataFrame df в файл data/df.csv
df = pd.read_csv('../data/df.csv')

In [ ]:
# преобразуем столбец datetime в формат datetime64[ns]
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
df.info()

In [ ]:
# дропаем ненужные колонки для обучения
#columns_to_drop = ['milliseconds', 'place', 'status', 'tsunami', 'data_type', 'country', 'timezone']
#df = df.drop(columns_to_drop, axis=1)
df = df.drop(['milliseconds', 'place', 'status', 'tsunami', 'data_type', 'country', 'timezone'], axis=1)
df[:10]

In [ ]:
X = df.drop(['magnitude'], axis=1)
y = df['magnitude']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=random_state)